In [90]:
import pandas as pd
import numpy as np

df = pd.read_excel('data.xlsx')

In [91]:
df.head()

,序号,来源详情,您宝宝的性别,您的宝宝的出生日期：,(1)您有几个孩子,(2)个孩子，这是您的第___个孩子,宝宝的主要抚养人,请问宝宝的家庭类型是,宝宝母亲的年龄（周岁）,宝宝母亲的文化程度:,...,拥有足量的与年龄适宜的玩具,对孩子任何事情都要干涉,给孩子的诉说、表达或申辩的机会,带孩子到商店、公园,赞赏孩子的好奇心，有意识培养孩子兴趣,无端向孩子发脾气,有意识帮助孩子认知事物，扩大词汇,保持孩子活动的房间清洁、整齐,对孩子的需求能够作出及时的回应.1,总分
0,1,NaN,男,2023-06-20,1,2,保姆阿姨,联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）,23,初中及以下,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),62
1,2,NaN,男,2020-06-15,2,2,父母,核心家庭（宝宝与父母一起居住）,30,本科,...,总是,总是,总是,总是,总是,很少,总是,总是,总是,240
2,3,NaN,女,2022-03-09,2,2,父母,联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）,30,大专,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),132
3,4,NaN,女,2022-12-14,2,2,父母,核心家庭（宝宝与父母一起居住）,19岁,初中及以下,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),98
4,5,NaN,男,2023-03-15,2,2,父母,核心家庭（宝宝与父母一起居住）,33,大专,...,(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),(跳过),160


In [92]:
df.shape

(52640, 142)

In [93]:
df.columns

Index(['序号', '来源详情', '您宝宝的性别', '您的宝宝的出生日期：', '(1)您有几个孩子', '(2)个孩子，这是您的第___个孩子',
       '宝宝的主要抚养人', '请问宝宝的家庭类型是', '宝宝母亲的年龄（周岁）', '宝宝母亲的文化程度:',
       ...
       '拥有足量的与年龄适宜的玩具', '对孩子任何事情都要干涉', '给孩子的诉说、表达或申辩的机会', '带孩子到商店、公园',
       '赞赏孩子的好奇心，有意识培养孩子兴趣', '无端向孩子发脾气', '有意识帮助孩子认知事物，扩大词汇', '保持孩子活动的房间清洁、整齐',
       '对孩子的需求能够作出及时的回应.1', '总分'],
      dtype='object', length=142)

In [94]:
df.iloc[0]

序号                            1
来源详情                        NaN
您宝宝的性别                        男
您的宝宝的出生日期：           2023-06-20
(1)您有几个孩子                     1
                        ...    
无端向孩子发脾气                   (跳过)
有意识帮助孩子认知事物，扩大词汇           (跳过)
保持孩子活动的房间清洁、整齐             (跳过)
对孩子的需求能够作出及时的回应.1          (跳过)
总分                           62
Name: 0, Length: 142, dtype: object

## 数据预处理

In [95]:
print('总实例数', df.shape[0])
nullValue = df.isnull().sum()
for i in range(len(nullValue)):
    if nullValue[i] != 0 :
        print(df.columns[i], '缺失:', nullValue[i])

总实例数 52640
来源详情 缺失: 43121


In [96]:
df.fillna(0, axis=1, inplace=True)
df.isnull().any()

序号                   False
来源详情                 False
您宝宝的性别               False
您的宝宝的出生日期：           False
(1)您有几个孩子            False
                     ...  
无端向孩子发脾气             False
有意识帮助孩子认知事物，扩大词汇     False
保持孩子活动的房间清洁、整齐       False
对孩子的需求能够作出及时的回应.1    False
总分                   False
Length: 142, dtype: bool

In [97]:
df_drop = df.drop(columns=['序号', '您的宝宝的出生日期：'], axis=1)
data = df_drop.drop('总分', axis=1)
target = df_drop['总分']

In [98]:
feature = data.columns

In [99]:
target.head()

0     62
1    240
2    132
3     98
4    160
Name: 总分, dtype: object

In [115]:
(df['(1)您有几个孩子'] == '1↗').sum()

0

In [102]:
for i in feature:
    print(df[i].unique())

[0 '直接访问']
['男' '女']
['1' '2' '3' '俩' '一' '两' '两个' '二' '三' '一个' '4' '有' '2？' '四' '②' '18'
 '两个孩子' '了21' '11' '¦' '妈妈' '=' '5' '三个' '有一' '，2' '0' '6' '，' 'l' '（2）'
 '双胞胎' '2⃣️' '五' '兩' '个' '没' '几' '1⃣️' '③' '2@' '二个' '两二' '2l' '张佳依' '1¦'
 '2个' '男' '你' '22' '丨' '33' '一个小孩' '第' 's1' '可爱' 'I' 'Ⅰ' '兩個' '孙千越' '-个'
 '第二' '倆' '100' '2020' '8']
['2' '1' '二' '3' '一' '第一' '1，2' '三' 'z' 'q' '双胞胎' 'v' '两' '第一个' '=′' 'b'
 '4' '2.' '四' '第二' '，' '3.4' 'a\u20062' '2，3' '②' '一个' '28' '=' '双' '①'
 '2一r' '2和3' '1一' '第1' '2-3' '了' '¦' '5' '01' '0' '6' '二个' 'I' '26' 'ou2'
 '2⃣️' '12' '2”' '第二个' '1”' '1--' '⼀' 'c' '11' '1个' '1，' '第三个' '女' '1＿'
 '2到3' '，1' '2\\3' '2、3' '1⃣️' '2.3' '不是' '2‘' 'l' '三个' '双胞套' '第三' '3，4，5'
 '-' '男' '1/2' '2/3' '我' '小宝' '1.2' '27' '@1' '1-2' '小女儿' '哥哥' '-1' '丨'
 '1↗' '1️⃣' '是' '2，' '1、2' '10' '第一胎' '＇' '双胞胎 2-3' '老二' 'Ⅰ' '乙' 'Z' '2/'
 '第' '贰' '俩' '‘' '101' '小' '，2' '女儿' '8' '吧']
['保姆阿姨' '父母' '外祖父母或祖父母' '其他〖母亲〗' '其他〖奶奶跟妈妈〗' '其他〖母亲、外祖母〗' '其他〖自力更生〗' '其他'
 '其他〖姑姑〗' '其他〖爷爷，爸爸〗' '其他〖自己带〗

In [105]:
dicList = [
    {'直接访问': 1},
    {'男':0, '女':1},
    {'1':1, '2':2, '3':3, '一':1, '二':2, '三':3, '俩':2, '两':2, '两个':2, '一个':1, '4':4,
     '四':4, '5':5, '五':5, '三个':3, '四个':4, '五个':5},
    {'1':1, '2':2, '3':3, '一':1, '二':2, '三':3, '俩':2, '两':2, '两个':2, '一个':1, '4':4,
     '四':4, '5':5, '五':5, '三个':3, '四个':4, '五个':5, '第一':1, '第一个':1, '第二':2, '第二个':2},
    {'保姆阿姨':0, '父母':1, '外祖父母或祖父母':2, '其他*':3},
    {'联合家庭（宝宝、父母、祖父母（或外祖父母）三代一起居住）':0}
]
# df.replace({'来源详情' : dicList[0]})

In [117]:
dict(zip(['x', 'y', 'z'], [1, 2, 3]))

{'x': 1, 'y': 2, 'z': 3}